In [1]:
import tensorflow as tf

from __future__ import  absolute_import
from __future__ import division
from __future__ import  print_function

import cifar10_input

data_dir = "../cifar_data/cifar-10-batches-bin/"
batch_size = 64

train_imgs, train_labels = cifar10_input.distorted_inputs(data_dir=data_dir, batch_size=batch_size)

val_images, val_labels = cifar10_input.inputs(eval_data=True,data_dir="../cifar_data/cifar-10-batches-bin/",
                                               batch_size=batch_size)

train_examples = cifar10_input.NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN
val_examples = cifar10_input.NUM_EXAMPLES_PER_EPOCH_FOR_EVAL

Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.FixedLengthRecordDataset`.
Filling queue with 20000 CIF

In [2]:
def variable_weight(shape, stddev=5e-2):
    init = tf.truncated_normal_initializer(stddev=stddev)
    return tf.get_variable(initializer=init, shape=shape, name='weight')

def variable_bias(shape):
    init = tf.constant_initializer(0.1)
    return tf.get_variable(initializer=init, shape=shape, name='bias')

def conv(x, ksize, out_depth, strides, padding='SAME', act=tf.nn.relu, scope='conv_layer', reuse=None):
    in_depth = x.get_shape().as_list()[-1]
    with tf.variable_scope(scope, reuse=reuse):
        shape = ksize+[in_depth, out_depth]
        with tf.variable_scope('kernel'):
            kernel = variable_weight(shape=shape)
        strides = [1, strides[0], strides[1],1]
        conv = tf.nn.conv2d(x, kernel, strides, padding, name='conv')
        with tf.variable_scope('bias'):
            bias = variable_bias([out_depth])
        preact = tf.nn.bias_add(conv,bias)
        out = act(preact)
        return out

def max_pool(x, ksize, strides, padding='SAME', name='pool_layer'):
    return tf.nn.max_pool(x,[1, ksize[0], ksize[1],1],[1,strides[0], strides[1],1], padding, name=name)

def fc(x, out_depth, act=tf.nn.relu, scope='fully_connect', reuse=None):
    in_depth = x.get_shape().as_list()[-1]
    with tf.variable_scope(scope, reuse=reuse):
        with tf.variable_scope('weight'):
            weitht = variable_weight([in_depth, out_depth])
        with tf.variable_scope('bias'):
            bias= variable_bias([out_depth])
        fc = tf.nn.bias_add(tf.matmul(x,weitht),bias, name='fc')
        out = act(fc)
        
        return out

def alexnet(inputs, reuse=None):
    with tf.variable_scope('AlexNet', reuse=reuse):
        #第一层， 5x5卷积， 64个卷积核， 步长1x1， padding 'VALID'   24*24
        net = conv(inputs, [5,5], 64, [1,1], padding="VALID", scope='conv1')
        # 第二层是 3x3 的池化, 步长是 2x2, padding是`VALID`    20*20
        net = max_pool(net, [3, 3], [2, 2], padding='VALID', name='pool1')
        #第三层                 10*10
        net = conv(net, [5,5], 64, [1,1], scope='conv2')
        #第四层         10*10
        net = max_pool(net, [3,3], [2,2], padding="VALID", name='pool2')
        
        #return net.get_shape()
        net = tf.reshape(net, [-1, 4*4*64])
        net = fc(net, 384, scope='fc3')
        net = fc(net, 192, scope='fc4')
        net = fc(net, 10, scope='fc5', act=tf.identity)
        return net

In [ ]:
import time
#定义输入数据的　placeholder
image_holder = tf.placeholder(tf.float32,[batch_size,24,24,3])
label_holder = tf.placeholder(tf.int32,[batch_size])

train_out = alexnet(image_holder)
def loss(logits, labels):
    labels = tf.cast(labels,tf.int64)
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=labels, 
                                                                name='cross_entropy_example')
    cross_entropy_mean = tf.reduce_mean(cross_entropy,name='cross_entropy')
    tf.add_to_collection('losses',cross_entropy_mean)
    return tf.add_n(tf.get_collection('losses'),name='total_loss')
loss = loss(train_out, label_holder)

train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)
#top_k_op = tf.nn.in_top_k(train_out, label_holder,1)
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()
tf.train.start_queue_runners()

for step in range(1000):
    start_time = time.time()
    image_batch, label_batch = sess.run([train_imgs, train_labels])
    _, loss_value = sess.run([train_op, loss],feed_dict={image_holder: image_batch, label_holder: label_batch})
    duration = time.time() - start_time
    if step % 10 ==0:
        examples_per_sec = batch_size/duration
        sec_per_batch = float(duration)
        
        format_str = ('step %d, loss=%.2f (%.1f examples/sec; %.3f sec/batch)')
        print(format_str % (step, loss_value, examples_per_sec, sec_per_batch))

In [ ]:
train_out = alexnet(train_imgs)
val_out = alexnet(val_images,reuse=True)

with tf.variable_scope('loss'):
    train_loss = tf.reduce_mean(tf.losses.sparse_softmax_cross_entropy(logits=train_out, labels=train_labels, scope='train'))
    val_loss = tf.reduce_mean(tf.losses.sparse_softmax_cross_entropy(logits=val_out, labels=val_labels, scope='val'))

with tf.name_scope('accuracy'):
    with tf.name_scope('train'):
        train_acc = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(train_out, axis=-1, output_type=tf.int32),
                                                    train_labels), tf.float32))
    with tf.name_scope('train'):
        val_acc = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(val_out, axis=-1, output_type=tf.int32),
                                                  val_labels), tf.float32))

lr = 0.01
opt = tf.train.MomentumOptimizer(learning_rate=lr, momentum=0.9)
train_op = opt.minimize(train_loss)
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
tf.train.start_queue_runners()

for i in range(20000):
    sess.run([train_imgs, train_labels])
    sess.run(train_op)
    if i % 400 == 99:
        sess.run([val_images,val_labels])
        train_acct, train_losst, val_losst, val_acct= sess.run([train_acc,train_loss,val_loss, val_acc])
        #print(train_acct, train_losst)
        print("Step {} trainloss: {:.6f} trainacc: {:.6f} val_loss: {:.6f} val_acc: {:.6f}".format(i+1,train_acct, train_losst, val_losst, val_acct))
    
print("train done !")
print("-"*40)

Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Step 100 trainloss: 0.281250 trainacc: 1.787735 val_loss: 1.814496 val_acc: 0.375000
Step 500 trainloss: 0.359375 trainacc: 1.671352 val_loss: 1.717936 val_acc: 0.453125
